# Testing whether GPU is working

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

# Which GPU we are using?

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8929471684488163242
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11320098816
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11738412412414534181
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

# Ram information

In [3]:
!cat /proc/meminfo

MemTotal:       13302920 kB
MemFree:         9018056 kB
MemAvailable:   11768164 kB
Buffers:          134760 kB
Cached:          2605720 kB
SwapCached:            0 kB
Active:          1627436 kB
Inactive:        2310672 kB
Active(anon):    1003304 kB
Inactive(anon):     2508 kB
Active(file):     624132 kB
Inactive(file):  2308164 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               540 kB
Writeback:             0 kB
AnonPages:       1197732 kB
Mapped:           697212 kB
Shmem:              3240 kB
KReclaimable:     118572 kB
Slab:             173912 kB
SReclaimable:     118572 kB
SUnreclaim:        55340 kB
KernelStack:        6304 kB
PageTables:        15120 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651460 kB
Committed_AS:    4392284 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       48120 kB
VmallocChunk:          0 kB
Percpu:          

# CPU information

In [5]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:

import pandas as pd

In [2]:
df=pd.read_csv("Churn_Modelling.csv")
X=df.iloc[:,3:13]
y=df.iloc[:,13]

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#creating dummy variables of categorical features
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [6]:
X=pd.concat([X,geography,gender],axis=1) #concatinating
X=X.drop(['Geography','Gender'],axis=1) #droping the unnecessary columns

In [7]:
#splitting the dataset in to train set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
#now lets make ANN
import keras
from keras.models import Sequential#sequential library is responsible for creating neural networks
from keras.layers import Dense #dense library is used to create the hidden layers
#from keras.layers import LeakyReLU,PReLU,ELU #
from keras.layers import Dropout #dropout is an regularization library for neural networks

In [10]:
#initializing the ANN,it is an empty neural network here
classifier=Sequential()

In [11]:
#adding the input layer and the 1st hidden layer using dense
#units specifies the number of neurons in the hidden layer
#kernel_initializer is the initialization of parameter of weights
#when we use relu activation function he_unform and he_normal works well
classifier.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu',input_dim=11))
classifier.add(Dropout(0.3))

In [12]:
#adding 2nd hidden layer
classifier.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dropout(0.4))

In [13]:
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [14]:
#compiling the ANN
#if our problem is binary classification we should use binarycrossentropy and if multiclass we should use categoricalcrossentropy
classifier.compile(optimizer='adamax',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
model_history=classifier.fit(X_train, y_train, batch_size = 10,validation_split=0.3,epochs = 100)

Epoch 1/100
800/800 [==============================] - 6s 3ms/step - loss: 0.5406 - accuracy: 0.7710
Epoch 2/100
800/800 [==============================] - 3s 3ms/step - loss: 0.4937 - accuracy: 0.7964
Epoch 3/100
800/800 [==============================] - 3s 3ms/step - loss: 0.4745 - accuracy: 0.7986
Epoch 4/100
800/800 [==============================] - 3s 4ms/step - loss: 0.4594 - accuracy: 0.8024
Epoch 5/100
800/800 [==============================] - 3s 3ms/step - loss: 0.4475 - accuracy: 0.8039
Epoch 6/100
800/800 [==============================] - 3s 3ms/step - loss: 0.4385 - accuracy: 0.8077
Epoch 7/100
800/800 [==============================] - 3s 3ms/step - loss: 0.4315 - accuracy: 0.8110
Epoch 8/100
800/800 [==============================] - 3s 3ms/step - loss: 0.4255 - accuracy: 0.8124
Epoch 9/100
800/800 [==============================] - 3s 3ms/step - loss: 0.4203 - accuracy: 0.8163
Epoch 10/100
800/800 [==============================] - 3s 3ms/step - loss: 0.4154 - accura

In [16]:
# predicting the results
y_pred=classifier.predict(X_test)
y_pred=(y_pred > 0.5)

In [18]:
#calculating the accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

In [19]:
print(score)

0.8595
